In [2]:
import numpy as np
from dataclasses import dataclass

from typing import List, Dict, Union


RAW = """7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7"""

@dataclass
class Board():
    """
    Board class for a bingo game
    A 5 x 5 board is instanciated
    """
    board:np.array
    row_count:np.array=np.zeros(shape=(5,))
    col_count:np.array=np.zeros(shape=(5,))
        
    
    def update_count(self, number:int):
        new_board = self.board.copy()
        new_board = np.where(new_board==number, np.nan, new_board)
        cond = np.isnan(new_board)
        self.row_count = np.nansum(cond, axis=1)
        self.col_count = np.nansum(cond, axis=0)
        self.board = new_board


class Bingo():
    """
    Plays Bing game for boards
    instantiated.
    """
    def __init__(self, raw):
        self.text = raw
        self._numbers: List[int] = self.parse_numbers(self.text)
        self.numbers:List[int] = self._numbers.copy()
        self.boards:List[Board] = self.parse_boards(self.text)

    def parse_numbers(self, text:str)->List[int]:
        numbers = text.splitlines()
        return [int(num) for num in numbers[0].split(",")][::-1]

    def parse_boards(self, text:str)->List[np.array]:
        data = text.splitlines()
        raw_boards = []
        for board in data[1:]:
            if board:
                raw_boards.append([int(num) for num in  board.split(" ") if num])
        boards  = np.array(raw_boards).reshape((len(raw_boards)//5, 5, 5))
        return [Board(board) for board in boards]
        
    def draw_number(self)->int:
        return self.numbers.pop() if self.numbers else []
        
    def score(self, board:Board, number:int)->int:
        new_board = board.board.copy()
        unmarked_sum = np.nansum(new_board)
        print(f'score is ', unmarked_sum, number)
        return unmarked_sum * number        
    
    
    def game_win_first(self)-> Union[int, str]:
        number = self.draw_number()
        #boards = []
        while isinstance(number, int):
            for board in self.boards:
                board.update_count(number=number)
                #boards.append([(number, board.board)])
                if np.any(board.row_count == 5) or np.any(board.col_count == 5):
                    return self.score(board=board, number=number)
            number = self.draw_number()
        #self.history = boards
        return "no winners found"
    
    def game_win_last(self)-> Union[int, str]:
        number = self.draw_number()
        last_board = len(self.boards)
        completed = set()
        while True:
            for i, board in enumerate(self.boards):
                if i in completed:
                    continue
                else:
                    board.update_count(number=number)
                if np.any(board.row_count == 5) or np.any(board.col_count == 5):
                    completed.add(i)
                    if len(completed) == last_board:
                        return self.score(board=board, number=number) 
            else:
                number = self.draw_number()

b = Bingo(raw=RAW)
assert b.game_win_first() == 4512

with open('inputs/day4.txt') as file:
    PUZZLE = file.read()
    B = Bingo(raw=PUZZLE)
    print('part1', B.game_win_first())
    print('part2', B.game_win_last())

score is  188.0 24
score is  947.0 44
part1 41668.0
score is  338.0 31
part2 10478.0
